In [46]:
import numpy as np
from pymatgen.core import Structure, Lattice, PeriodicSite, DummySpecie, Species
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import MinimumDistanceNN, CrystalNN, VoronoiNN, JmolNN

In [ ]:
'''# Load defective structure
defective_struct = Structure.from_file("low_density_defects/MoS2/MoS2cifs/6141cf1a1d648932fbc34297.cif")
struct_lattice = defective_struct.lattice

# Get reference structure
ref_unit_cell = Structure.from_file("low_density_defects/MoS2/MoS2.cif")
reference_struct = ref_unit_cell.make_supercell([8,8,1])'''


/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/core/structure.py:3087: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 48 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [47]:
# Load defective structure
defective_struct = Structure.from_file("/home/adamkim/PROGRAMMING/Dataset/high_density_defects/MoS2_500/highMoS2cifs/MoS2_Mo56W4Se3S117_b593cd47-47e7-4a55-9a27-466f5c3f661c.cif")
struct_lattice = defective_struct.lattice

# Get reference structure
ref_unit_cell = Structure.from_file("/home/adamkim/PROGRAMMING/Dataset/high_density_defects/MoS2_500/MoS2.cif")
reference_struct = ref_unit_cell.make_supercell([8,8,1])


/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/core/structure.py:3087: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 45 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [48]:
def struct_to_dict(structure):
    list_of_sites = structure.sites
    list_of_frac_coords = np.round(structure.frac_coords,3)
    structure_dict = {i: j for i, j in zip(list_of_sites, list_of_frac_coords)}
    return structure_dict

In [ ]:
def get_defect_structure(defective_dict, reference_dict, struct_lattice): 
    defect_site = []

    for ref_site, ref_coords in reference_dict.items():
        matching = False
        for def_site, def_coords in defective_dict.items():
            if np.array_equal(ref_coords, def_coords):
                matching = True
                if ref_site.specie != def_site.specie:  # Substitution case
                    defect_site.append(PeriodicSite(
                        species= def_site.species,
                        coords= def_site.frac_coords,
                        coords_are_cartesian=False,
                        lattice= struct_lattice,
                        properties= {"original_new_am": (ref_site.specie.Z, def_site.specie.Z)}
                    ))

        if not matching:           # Vacancy case
            defect_site.append(PeriodicSite(
                species=DummySpecie(),
                coords = ref_coords,
                coords_are_cartesian=False,
                lattice = struct_lattice,
                properties = {"original_new_am": (ref_site.specie.Z, 0)}
                ))

    # Create a structure with the defect sites
    defect_struct = Structure.from_sites(defect_site)
    return defect_struct

# Convert structures to dictionaries
defective_dict = struct_to_dict(defective_struct)
reference_dict = struct_to_dict(reference_struct)

# Get defect structure
defect_struct = get_defect_structure(defective_dict, reference_dict, struct_lattice)


In [ ]:
def get_defects(defective_struct, reference_struct):
    copy_defective_struct = defective_struct.copy()
    # struct to dict
    defective_dict = struct_to_dict(copy_defective_struct)
    reference_dict = struct_to_dict(reference_struct)

    # Get lattice of defective structure
    structure_lattice = copy_defective_struct.lattice

    # Define list to add all defect sites
    defects_list = []

    # Define a dictionary to hold atomic numbers
    sample_property = {}

    for ref_site, ref_coords in reference_dict.items():
        matching = False
        for def_site, def_coords in defective_dict.items():
            if np.array_equal(ref_coords, def_coords):
                matching = True
                if ref_site.specie != def_site.specie: # Substitution case
                    # Add site to defects list
                    defects_list.append(def_site)

                    # Get atomic number change
                    am_property = {"orig_new_am":(ref_site.specie.Z, def_site.specie.Z)}
                    sample_property[def_site] = am_property

                    

        if not matching: # Vacancy case
            # Add site to defective structure
            vacant_site = PeriodicSite(
                species= DummySpecie("X"),
                coords=ref_coords,
                coords_are_cartesian=False, 
                lattice=structure_lattice
                )
            
            copy_defective_struct.append(vacant_site.species, vacant_site.coords)

            # Add site to defects list
            defects_list.append(vacant_site)

            # Get atomic number change
            am_property={"orig_new_am": (ref_site.specie.Z, 0)}
            sample_property[vacant_site] = am_property

            

    # Till this point every defect site is in the defective structure
    return copy_defective_struct, defects_list, sample_property


# Get defect structure
new_defective_struct, to_compare, the_am_property = get_defects(defective_struct, reference_struct)


In [ ]:
print(new_defective_struct)

Let's get the nearest neigbors of the defects in the defective structure

## Using `MinimumDistanceNN`

In [ ]:
def get_properties(a_defect_site, all_defect_sites, am_property):
    # Get index of defect site
    list_of_sites = new_defective_struct.sites
    the_index = list_of_sites.index(a_defect_site)

    # Define an instance responsible for getting the nearest neighbors
    cnn = MinimumDistanceNN(cutoff=3.0)

    # Get the nearest neighbors of each defect site
    neighbors = cnn.get_nn_info(new_defective_struct, the_index)

    # Get the coordination numbers
    coord_numbers = len(neighbors)

    # Get distance to neighbors and atomic numbers
    bl_to_neighbors = []
    elements_am = []
    for neighbor_info in neighbors:
        bond_length = neighbor_info['weight'] 
        the_am = neighbor_info["site"].specie.Z
        if the_am > 0:
            elements_am.append(the_am)
        else:
            the_am = 0
            elements_am.append(the_am)
            bl_to_neighbors.append(bond_length)
        

    # Get distance to other defect sites
    dist_to_other_defects = []
    for n in all_defect_sites:
        if a_defect_site == n:
            pass
        else:
            dist = a_defect_site.distance(n)
            dist_to_other_defects.append(dist)

    # Collect these properties as dictionaries
    new_dict = {"nearest_neighbors": neighbors,
                "neighbors_atomic_numbers": elements_am,
                "bond_length_to_neighbors": bl_to_neighbors,
                "coordination_number": coord_numbers,
                "bond_length_to_defects": dist_to_other_defects}
    
    return new_dict

for i in to_compare:
    sampler = get_properties(i,to_compare)
    print(sampler)
    print(" ")

{'nearest_neighbors': [{'site': PeriodicNeighbor: S119 (S) (-12.76, 20.26, 2.155) [-0.04167, 0.9167, 0.1077], 'image': (np.float64(-1.0), np.float64(0.0), np.float64(0.0)), 'weight': np.float64(0.9928990871634384), 'site_index': np.int64(119)}, {'site': PeriodicNeighbor: S179 (S) (-12.76, 20.26, 5.285) [-0.04167, 0.9167, 0.2642], 'image': (np.float64(-1.0), np.float64(0.0), np.float64(0.0)), 'weight': np.float64(0.9930309385836875), 'site_index': np.int64(179)}, {'site': PeriodicNeighbor: S63 (S) (-11.17, 23.02, 2.155) [0.08333, 1.042, 0.1077], 'image': (np.float64(0.0), np.float64(1.0), np.float64(0.0)), 'weight': np.float64(0.9928990848788402), 'site_index': np.int64(63)}, {'site': PeriodicNeighbor: S120 (S) (-11.17, 23.02, 5.285) [0.08333, 1.042, 0.2642], 'image': (np.float64(0.0), np.float64(1.0), np.float64(0.0)), 'weight': np.float64(0.9930309362981786), 'site_index': np.int64(120)}, {'site': PeriodicNeighbor: S68 (S) (-9.571, 20.26, 2.155) [0.08333, 0.9167, 0.1077], 'image': (np

In [ ]:
def main():
    # Load defective structure
    defective_struct = Structure.from_file("/home/adamkim/PROGRAMMING/Dataset/high_density_defects/MoS2_500/highMoS2cifs/MoS2_Mo56W4Se3S117_b593cd47-47e7-4a55-9a27-466f5c3f661c.cif")
    struct_lattice = defective_struct.lattice

    # Get reference structure
    ref_unit_cell = Structure.from_file("/home/adamkim/PROGRAMMING/Dataset/high_density_defects/MoS2_500/MoS2.cif")
    reference_struct = ref_unit_cell.make_supercell([8,8,1])
    reference_struct = ref_unit_cell.make_supercell([8,8,1])

    # Get defect structure
    new_defective_struct, the_defect_sites,to_compare = get_defects(defective_struct, reference_struct)

    # Get the nearest neighbors of each site
    cnn = CrystalNN()
    list_of_sites = new_defective_struct.sites
    list_of_neighbors = []

    # Get the nearest neighbors of each defect site
    for defect_site in to_compare:
        the_index = list_of_sites.index(defect_site)
        neighbors = cnn.get_nn(new_defective_struct, the_index)
        list_of_neighbors.append(neighbors)

    print(list_of_neighbors)


def struct_to_dict(structure):
    list_of_sites = structure.sites
    list_of_frac_coords = np.round(structure.frac_coords,3)
    structure_dict = {i: j for i, j in zip(list_of_sites, list_of_frac_coords)}
    return structure_dict

def get_defects(defective_struct, reference_struct):
    copy_defective_struct = defective_struct.copy()
    # struct to dict
    defective_dict = struct_to_dict(copy_defective_struct)
    reference_dict = struct_to_dict(reference_struct)

    # Get lattice of defective structure
    structure_lattice = copy_defective_struct.lattice

    # Define list to add all defect sites
    defect_sites_to_struct = []
    for_index_comparison = []

    for ref_site, ref_coords in reference_dict.items():
        matching = False
        for def_site, def_coords in defective_dict.items():
            if np.array_equal(ref_coords, def_coords):
                matching = True
                if ref_site.specie != def_site.specie:  # Substitution case
                    for_index_comparison.append(def_site)

                    sub_site = PeriodicSite(
                        species=def_site.specie,
                        coords= def_site.frac_coords,
                        coords_are_cartesian= False,
                        lattice= structure_lattice,
                        properties= {"orig_new_am":(ref_site.specie.Z, def_site.specie.Z)}
                    )
                    defect_sites_to_struct.append(sub_site)

        if not matching:           # Vacancy case
            # Add site to defective structure
            vacant_specie = DummySpecie(symbol="X")
            the_coords = ref_coords
            copy_defective_struct.append(vacant_specie, the_coords)

            for_index_comparison.append(PeriodicSite(
                species= vacant_specie,
                coords=the_coords,
                coords_are_cartesian=False, 
                lattice=structure_lattice
                ))

            # Create a proper vacant site
            vacant_site = PeriodicSite(
                species= vacant_specie,
                coords=the_coords,
                coords_are_cartesian=False, 
                lattice=structure_lattice,
                properties={"orig_new_am": (ref_site.specie.Z, 0)}
                )
            defect_sites_to_struct.append(vacant_site)
            

    # Till this point every defect site is in the defective structure
    return copy_defective_struct, defect_sites_to_struct, for_index_comparison

if __name__ == "__main__":
    main()


# To Do
Add the `am_property` to the `get_property` function.